<a href="https://colab.research.google.com/github/inggitrestuillahi/Ensemble-Learning/blob/main/tugas3_seed_1301200307.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown '1w83l0drLxWM-58Y4G1ymhi41Jz1G91uK'

Downloading...
From: https://drive.google.com/uc?id=1w83l0drLxWM-58Y4G1ymhi41Jz1G91uK
To: /content/seeds_dataset.csv
100% 9.37k/9.37k [00:00<00:00, 9.12MB/s]


In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## Isi nama dan NIM seluruh anggota kelompok di sini
#   Nama                        NIM
#   Adhyasta Naufal Faadhilah   1301200321
#   Fikri Maulana               1301200239
#   Inggit Restu Illahi         1301204446
#   Rangga Eka Permana          1301200307

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree

# Load Dataset
Inisialisasi variable, masukan NIM di sini untuk random state.

Gunakan NIM salah satu anggota kelompok.



In [ ]:
filepath = "seeds_dataset.csv"
dot_filename = 'tree.dot'
# 1301200307
nim = 1301200307

Load dataset, cek konten data

Target kelas: **Category** 

In [ ]:
df = pd.read_csv(filepath, sep="\t")
df.sample(10, random_state=nim)

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len,Category
77,20.71,17.23,0.8763,6.579,3.814,4.451,6.451,2
70,17.63,15.98,0.8673,6.191,3.561,4.076,6.060,2
63,13.22,13.84,0.8680,5.395,3.070,4.157,5.088,1
28,14.11,14.18,0.8820,5.541,3.221,2.754,5.038,1
132,15.38,14.90,0.8706,5.884,3.268,4.462,5.795,2
134,15.56,14.89,0.8823,5.776,3.408,4.972,5.847,2
144,11.82,13.40,0.8274,5.314,2.777,4.471,5.178,3
182,12.19,13.36,0.8579,5.240,2.909,4.857,5.158,3
13,13.78,14.06,0.8759,5.479,3.156,3.136,4.872,1
82,20.20,16.89,0.8894,6.285,3.864,5.173,6.187,2


Training/test split **60/40**

In [ ]:
# X: semua kolom, Y: kolom terakhir (category)
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.6, random_state = nim)

# We perform bootstrap on the train data
# Keep test data for final accuracy testing
len(x_train), len(x_test)

(126, 84)

# Random Forest
Build a Random Forest, consisting of 20 Decision Tree classifer, with the following steps:

1. Prepare Bootstrap data (choose randomly from the *training data*, use sampling with replacement)
2. Choose 2-4 random features/atributes for each tree
3. Build Decision Tree classifier using step (1) and (2)
4. Calculate OOB score within the tree only

**NOTE**
This whole exercise has been simplified.
- Tree can be created using a library, therefore you may not be able control the feature selection on node level
- Random features are only selected in the beginning of each tree.
- OOB score is only calculated within its own tree instead.
- Typically Random Forest don't use train/test split and can be validated using OOB error. We simplify this to make it easier to test.
- You are not allowed to use a Random Forest or Bagging library.


In [ ]:
np.random.seed(nim)
n_tree = 20

forest = []
bootstrap_columns = []

# Iterate over the desired number of trees
for i in range(n_tree):
    # Select a random subset of the training data to use as the bootstrapped dataset for this tree
    rows = np.random.choice(len(x_train), len(x_train), replace=True)
    x_bootstrap = x_train.iloc[rows]
    y_bootstrap = y_train.iloc[rows]
    
    # Out of Bag data
    non_selected_rows = np.setdiff1d(np.arange(len(x_train)), rows)
    x_oob = x_train.iloc[non_selected_rows]
    y_oob = y_train.iloc[non_selected_rows]
    
    # Select a random subset of the features/attributes to use as the candidates for splitting nodes in this tree
    # Check for any repeated combinations
    feature_taken = np.random.randint(2, 4)
    cols = np.random.choice(x_bootstrap.columns, feature_taken, replace=False)
    bootstrap_columns.append(cols)
    
    # Filter the bootstrapped dataset based on the selected columns
    x_bootstrap = x_bootstrap[cols]

    # Build a decision tree classifier using the bootstrapped dataset and the selected features
    tree_model = tree.DecisionTreeClassifier(random_state=nim)
    tree_model.fit(x_bootstrap, y_bootstrap)


    # Add it to the forest
    forest.append(tree_model)
    
    # Calculate the "out-of-bag" (OOB) score for this tree using the non-selected rows from the bootstrapped dataset as the test set and the predictions from the tree as the predicted labels
    oob_score = tree_model.score(x_oob[cols], y_oob)
    print(f"{cols}" ,oob_score)


['Perimeter' 'Compactness'] 0.7647058823529411
['Kernel_width' 'Groove_len'] 0.8723404255319149
['Kernel_len' 'Groove_len' 'Area'] 0.9375
['Kernel_width' 'Compactness'] 0.7954545454545454
['Assymetry' 'Kernel_len' 'Kernel_width'] 0.9361702127659575
['Perimeter' 'Compactness'] 0.7272727272727273
['Assymetry' 'Kernel_len' 'Kernel_width'] 0.9069767441860465
['Kernel_width' 'Assymetry'] 0.8571428571428571
['Compactness' 'Groove_len' 'Assymetry'] 0.8
['Groove_len' 'Kernel_len'] 0.9318181818181818
['Area' 'Kernel_width'] 0.775
['Assymetry' 'Perimeter' 'Groove_len'] 0.8780487804878049
['Kernel_width' 'Assymetry' 'Compactness'] 0.7636363636363637
['Compactness' 'Kernel_width'] 0.7674418604651163
['Kernel_len' 'Kernel_width'] 0.7674418604651163
['Kernel_len' 'Groove_len' 'Area'] 0.8333333333333334
['Kernel_len' 'Groove_len' 'Perimeter'] 0.9130434782608695
['Compactness' 'Kernel_width'] 0.717391304347826
['Perimeter' 'Assymetry' 'Kernel_len'] 0.8085106382978723
['Kernel_len' 'Groove_len' 'Compac

# Prediction and Voting
Run prediction on x_test data for each tree

Also calculate accuracy per tree here

In [ ]:
all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test[bootstrap_columns[i]]

    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...
    acc = (predictions == y_test).mean()

    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)

### Perform voting on the predictions from the Random Forest

all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test[bootstrap_columns[i]]

    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...

    acc = (predictions == y_test).mean()
    print(f'{bootstrap_columns[i]} : {acc}')

    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)